In [11]:
import time
import taskingai
# Load TaskingAI API Key fron environment variable

# TaskingAI: Chat with Assistant Example

In this example, we will first create an assistant who knows the meaning of various numbers and will explain it in certain language.
Then we will start a chat with the assistant.

##  Create Assistant

In [2]:
from taskingai.tool import Action, ActionAuthentication, ActionAuthenticationType
from typing import List

# create an assistant action
NUMBERS_API_SCHEMA = {
        "openapi": "3.0.0",
        "info": {
            "title": "Numbers API",
            "version": "1.0.0",
            "description": "API for fetching interesting number facts"
        },
        "servers": [
            {
            "url": "http://numbersapi.com"
            }
        ],
        "paths": {
            "/{number}": {
            "get": {
                "summary": "Get fact about a number",
                "parameters": [
                {
                    "name": "number",
                    "in": "path",
                    "required": True,
                    "description": "The number to get the fact for",
                    "schema": {
                    "type": "integer"
                    }
                }
                ],
                "responses": {
                "200": {
                    "description": "A fact about the number",
                    "content": {
                    "text/plain": {
                        "schema": {
                        "type": "string"
                        }
                    }
                    }
                }
                }
            }
        }
    }
}
actions: List[Action] = taskingai.tool.bulk_create_actions(
    schema=NUMBERS_API_SCHEMA,
    authentication=ActionAuthentication(
        type=ActionAuthenticationType.NONE,
    )
)
action = actions[0]
print(f"created action: {action}\n")

created action: {'action_id': 'bFBd2uQJa2wp8ac6kEDLE4yt',
 'authentication': {'content': None, 'secret': None, 'type': 'none'},
 'created_timestamp': 1701160841935,
 'object': 'Action',
 'schema': {'components': {},
            'info': {'description': 'API for fetching interesting number facts',
                     'title': 'Numbers API',
                     'version': '1.0.0'},
            'openapi': '3.0.0',
            'paths': {'/{number}': {'get': {'parameters': [{'description': 'The '
                                                                           'number '
                                                                           'to '
                                                                           'get '
                                                                           'the '
                                                                           'fact '
                                                                           'for',
       

In [3]:
from taskingai.assistant import Assistant, Chat, AssistantTool, AssistantToolType

# choose an available chat_completion model from your project
model_id = "Gk1145Bl"

assistant: Assistant = taskingai.assistant.create_assistant(
    model_id=model_id,
    name="My Assistant",
    description="A assistant who knows the meaning of various numbers.",
    system_prompt_template=[
        "You know the meaning of various numbers.",
        "No matter what the user's language is, you will use the {{langugae}} to explain."
    ],
    tools=[AssistantTool(
        type=AssistantToolType.action,
        id=action.action_id,
    )],
    retrievals=[],
    metadata={"foo": "bar"},
)
print(f"created assistant: {assistant}\n")

created assistant: {'assistant_id': 'X5lMxXkQZahUAzsdIUCEidCH',
 'created_timestamp': 1701160845645,
 'description': 'A assistant who knows the meaning of various numbers.',
 'metadata': {'foo': 'bar'},
 'model_id': 'Gk1145Bl',
 'name': 'My Assistant',
 'object': 'Assistant',
 'retrievals': [],
 'system_prompt_template': ['You know the meaning of various numbers.',
                            "No matter what the user's language is, you will "
                            'use the {{langugae}} to explain.'],
 'tools': [{'id': 'bFBd2uQJa2wp8ac6kEDLE4yt', 'type': 'action'}]}


## Start a Chat 

In [14]:
chat: Chat = taskingai.assistant.create_chat(
    assistant_id=assistant.assistant_id,
)
print(f"created chat: {chat.chat_id}\n")

created chat: SdELbhZeozXgPdr4fXSE7pjG


In [15]:
from taskingai.assistant import Message

user_input = input("User Input: ")
while user_input != "q":
    # create user message
    taskingai.assistant.create_user_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        text=user_input,
    )
    print(f"User: {user_input}")
    
    # generate assistant response
    assistant_message: Message = taskingai.assistant.generate_assistant_message(
        assistant_id=assistant.assistant_id,
        chat_id=chat.chat_id,
        system_prompt_variables={
            "language": "Spanish"
        }
    )
    print(f"Assistant: {assistant_message.content['text']}")
    time.sleep(2)
    user_input = input("User: ")

User: Hello
Assistant: Hi there! How can I assist you today?
User: What is the meaning of 120
Assistant: The number 120 represents the drop in meters of the world's tallest freefall ride, The Giant Drop, located in Australia.
User: One more meaning
Assistant: Another meaning of the number 120 is that it is a file medium format developed by Kodak.
User: thanks
Assistant: You're welcome! If you have any more questions, feel free to ask.


In [18]:
# list messages
messages = taskingai.assistant.list_messages(
    assistant_id=assistant.assistant_id,
    chat_id=chat.chat_id,
)
for message in messages:
    print(f"{message.role}: {message.content['text']}")

assistant: You're welcome! If you have any more questions, feel free to ask.
user: thanks
assistant: Another meaning of the number 120 is that it is a file medium format developed by Kodak.
user: One more meaning
assistant: The number 120 represents the drop in meters of the world's tallest freefall ride, The Giant Drop, located in Australia.
user: What is the meaning of 120
assistant: Hi there! How can I assist you today?
user: Hello
